In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/home/dyuthi/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/dyuthi/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
deliveries_df = pd.read_csv('deliveries.csv')

In [3]:
deliveries_df

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260915,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,5,SS Iyer,AK Markram,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260916,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,9,6,VR Iyer,AK Markram,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260917,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,0,1,NaN,0,NaN,NaN,NaN
260918,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,0,1,NaN,0,NaN,NaN,NaN


In [4]:
deliveries_df.isna().sum()

# this is fine, we aren't using those columns

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batter                   0
bowler                   0
non_striker              0
batsman_runs             0
extra_runs               0
total_runs               0
extras_type         246795
is_wicket                0
player_dismissed    247970
dismissal_kind      247970
fielder             251566
dtype: int64

## Performance index for batsmen

In [69]:
df = deliveries_df.groupby(by=['batter', 'bowler']).agg({'total_runs':'sum', 'is_wicket':'sum'}).reset_index()

In [70]:
df.dropna(inplace=True)

In [71]:
df

,batter,bowler,total_runs,is_wicket
0,A Ashish Reddy,A Nehra,8,1
1,A Ashish Reddy,AB Dinda,9,0
2,A Ashish Reddy,AD Mathews,25,0
3,A Ashish Reddy,AD Russell,4,1
4,A Ashish Reddy,Anureet Singh,2,0
...,...,...,...,...
27876,Z Khan,SR Watson,4,0
27877,Z Khan,Shakib Al Hasan,3,0
27878,Z Khan,Sohail Tanvir,3,0
27879,Z Khan,VRV Singh,1,0


In [72]:
df.is_wicket.value_counts()

is_wicket
0    17785
1     8067
2     1472
3      376
4      129
5       29
6       13
7        9
9        1
Name: count, dtype: int64

In [73]:
df['batting_average'] = df['total_runs']/df['is_wicket']

In [74]:
df['batting_average']

0        8.0
1        inf
2        inf
3        4.0
4        inf
        ... 
27876    inf
27877    inf
27878    inf
27879    inf
27880    inf
Name: batting_average, Length: 27881, dtype: float64

In [75]:
temp_df = df[~np.isinf(df['batting_average'])]

In [76]:
best_per_batsman = temp_df.groupby(by=['batter']).agg({'batting_average':'mean'})

In [77]:
best_per_batsman

,batting_average
batter,
A Ashish Reddy,4.892857
A Badoni,8.901515
A Chandila,0.000000
A Chopra,3.200000
A Choudhary,4.000000
...,...
Yashpal Singh,4.750000
Younis Khan,1.000000
Yudhvir Singh,1.750000


In [78]:
mapping = best_per_batsman['batting_average'].to_dict()

In [79]:
df['batting_average'] = np.where(
    df['batting_average'] == np.inf,
    df['batter'].map(mapping),
    df['batting_average']
)


In [80]:
df.head(5)

,batter,bowler,total_runs,is_wicket,batting_average
0,A Ashish Reddy,A Nehra,8,1,8.000000
1,A Ashish Reddy,AB Dinda,9,0,4.892857
2,A Ashish Reddy,AD Mathews,25,0,4.892857
3,A Ashish Reddy,AD Russell,4,1,4.000000
4,A Ashish Reddy,Anureet Singh,2,0,4.892857


In [81]:
df.dropna(inplace=True)

In [82]:
df

,batter,bowler,total_runs,is_wicket,batting_average
0,A Ashish Reddy,A Nehra,8,1,8.000000
1,A Ashish Reddy,AB Dinda,9,0,4.892857
2,A Ashish Reddy,AD Mathews,25,0,4.892857
3,A Ashish Reddy,AD Russell,4,1,4.000000
4,A Ashish Reddy,Anureet Singh,2,0,4.892857
...,...,...,...,...,...
27876,Z Khan,SR Watson,4,0,1.769231
27877,Z Khan,Shakib Al Hasan,3,0,1.769231
27878,Z Khan,Sohail Tanvir,3,0,1.769231
27879,Z Khan,VRV Singh,1,0,1.769231


In [83]:
avg_pi_batsmen = df.groupby('batter').agg({'batting_average':'mean'}).reset_index()

In [84]:
avg_pi_batsmen[avg_pi_batsmen['batter'] == 'MS Dhoni']  

,batter,batting_average
349,MS Dhoni,22.622935


In [85]:
avg_pi_batsmen['batting_average'].describe()

count    623.000000
mean       5.937045
std        5.857033
min        0.000000
25%        1.620192
50%        4.200000
75%        8.436508
max       30.420611
Name: batting_average, dtype: float64

In [87]:
avg_pi_batsmen[avg_pi_batsmen['batting_average'] == avg_pi_batsmen['batting_average'].max()]

,batter,batting_average
584,V Kohli,30.420611
